In [3]:
#ライブラリimport
import urllib, requests, regex, csv, re, time, socks, socket, sys, math, time, datetime, random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.request import urlopen


#スクレイピング後の待機時間を定義
scrayping_wait_time = 10

#プロキシ設定
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference('network.proxy.type', 1)
firefox_profile.set_preference('network.proxy.socks', '127.0.0.1')
firefox_profile.set_preference('network.proxy.socks_port', 9150)

#Firefoxのパスを引数に指定
global driver
driver = webdriver.Firefox(firefox_profile=firefox_profile)

#確認くん
driver.get("https://www.ugtop.com/spill.shtml")
time.sleep(scrayping_wait_time)

#################################
#格納用CSVを生成
#################################

#CSVファイルを読み込み
input_file = "Indeed_Scrayping_" + str(datetime.date.today()) + ".csv"

#CSVがなければ、ヘッダーを作成
try:
    input_df = pd.read_csv(input_file,encoding='cp932')
except:
    #各項目を格納する用の配列
    job_detail_url_list = []
    company_name_list = []
    work_location_list = []
    empstatus_list = []
    salary_list = []
    
    #各項目のヘッダーを格納
    job_detail_url_list.append("求人詳細URL")
    company_name_list.append("企業名")
    work_location_list.append("勤務地")
    empstatus_list.append("雇用形態")
    salary_list.append("給与")

    #CSVに出力する
    data = zip(job_detail_url_list,company_name_list,work_location_list,empstatus_list,salary_list)
    with open(input_file,'a',errors='backslashreplace',encoding="SHIFT-JIS") as fout:
        writecsv = csv.writer(fout,lineterminator='\n')
        writecsv.writerows(data)

#キーワードのエンコード
keyword = "エンジニア"
keywords = urllib.parse.quote(keyword)

time.sleep(scrayping_wait_time)

#ページのパラメータの初期値
page_parameter = 0
#ページのパラメータの上限を設定（Max100ページ）
page_parameter_max= 990

#Maxの100ページまで繰り返し
for i in range(page_parameter_max):
    
    #詳細URLリスト
    job_detail_url_list = []
    #会社名リスト
    company_name_list = []
    #勤務地リスト
    work_location_list = []
    #雇用形態リスト
    empstatus_list = []
    #給与リスト
    salary_list = []
    
    #求人一覧ページ
    job_list_page_url = "https://jp.indeed.com/jobs?q=" + keywords + "&start=" + str(page_parameter)

    #指定したURLに遷移
    driver.get(job_list_page_url)
    time.sleep(scrayping_wait_time)

    #Beautifulsoupで求人一覧URLを取得
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(scrayping_wait_time)

    for maintable in soup.findAll(class_="jobsearch-SerpJobCard unifiedRow row result clickcard"):
        
        #「詳細URL」を取得して格納
        try:
            table = maintable.find("h2")
            parts = table.find("a")
            job_detail_url_list.append("https://jp.indeed.com" + parts.get("href"))
        except AttributeError:
            job_detail_url_list.append("情報なし")

        #「会社名」を取得して格納
        try:
            table = maintable.find(class_="company")
            company_name = re.sub("\r|\t|\n|\u3000| ","",table.text)
            company_name_list.append(company_name)
        except AttributeError:
            company_name_list.append("情報なし")
            
        #「勤務地」を取得して格納
        try:
            table = maintable.find(class_="location accessible-contrast-color-location")
            work_location = re.sub("\r|\t|\n|\u3000| ","",table.text)
            work_location_list.append(work_location)
        except AttributeError:
            work_location_list.append("情報なし")

        #「雇用形態」を取得して格納
        try:
            table = maintable.find(class_="jobTypeLabelsWrapper")
            empstatus = re.sub("\r|\t|\n|\u3000| ","",table.text)
            empstatus_list.append(empstatus)
        except AttributeError:
            empstatus_list.append("情報なし")

        #「給与」を取得して格納
        try:
            table = maintable.find(class_="salaryText")
            salary = re.sub("\r|\t|\n|\u3000| ","",table.text)
            salary_list.append(salary)
        except AttributeError:
            salary_list.append("情報なし")

    #CSVに出力する　※一旦、csvの保存先はこのファイルと同階層で設定。後で../99_Outputへつなぐ
    data = zip(job_detail_url_list,company_name_list,work_location_list,empstatus_list,salary_list)
    with open("Indeed_Scrayping_" + str(datetime.date.today()) + ".csv",'a',errors='backslashreplace',encoding="SHIFT-JIS") as fout:
        writecsv = csv.writer(fout,lineterminator='\n')
        writecsv.writerows(data)
    
    #ページのパラメータを10増やす
    page_parameter += 10

    print("{}ページ目が完了しました。".format(i+1))
    
    #100件目以降は表示されないため、breakしてブラウザを閉じる
    if page_parameter >= 10:
        break

print("スクレイピングが完了しました")
driver.close()

スクレイピングが完了しました
